In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/train', './input/test', './input/weather_train', './input/weather_test',
                './input/building_metadata']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# FIRST-AUTHOR: remove path printing
# import os
# for dirname, _, filenames in os.walk('./input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
# FIRST-AUTHOR: remove path printing
# import os
# os.getcwd()

In [3]:
# FIRST-AUTHOR: remove sys calls
# os.chdir("./input")

In [4]:
# FIRST-AUTHOR: remove ML code
# from sklearn.preprocessing import LabelEncoder
# from sklearn import metrics

In [5]:
# FIRST-AUTHOR: remove ML code
# from sklearn.metrics import mean_squared_error
# import lightgbm as lgb
# from sklearn.model_selection import train_test_split

In [6]:
#########Reading all the files#########
train_df=pd.read_csv('./input/train.scaled.csv')
test_df=pd.read_csv('./input/test.scaled.csv')
weather_train_df=pd.read_csv('./input/weather_train.scaled.csv')
weather_test_df=pd.read_csv('./input/weather_test.scaled.csv')
building_meta_df=pd.read_csv('./input/building_metadata.scaled.csv')

In [7]:
train_df.shape 
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [8]:
######### Converting time stamp to datetime formats##########
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'])
    
building_meta_df['primary_use'] = building_meta_df['primary_use'].astype('category')

In [9]:
########### Merging data frames - step1 #################
temp_df = train_df[['building_id']]
temp_df = temp_df.merge(building_meta_df, on=['building_id'], how='left')
del temp_df['building_id']
train_df = pd.concat([train_df, temp_df], axis=1)

temp_df = test_df[['building_id']]
temp_df = temp_df.merge(building_meta_df, on=['building_id'], how='left')

del temp_df['building_id']
test_df = pd.concat([test_df, temp_df], axis=1)
del temp_df, building_meta_df

In [10]:
############Merging data frames step-2 ################
temp_df = train_df[['site_id','timestamp']]
temp_df = temp_df.merge(weather_train_df, on=['site_id','timestamp'], how='left')

del temp_df['site_id'], temp_df['timestamp']
train_df = pd.concat([train_df, temp_df], axis=1)

temp_df = test_df[['site_id','timestamp']]
temp_df = temp_df.merge(weather_test_df, on=['site_id','timestamp'], how='left')

del temp_df['site_id'], temp_df['timestamp']
test_df = pd.concat([test_df, temp_df], axis=1)

del temp_df, weather_train_df, weather_test_df

In [11]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [12]:
test_df.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01,0,Education,7432,2008.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,1,1,0,2017-01-01,0,Education,2720,2004.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
2,2,2,0,2017-01-01,0,Education,5376,1991.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
3,3,3,0,2017-01-01,0,Education,23685,2002.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
4,4,4,0,2017-01-01,0,Education,116607,1975.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6


In [13]:
train_df.shape

(500000, 16)

In [14]:
evaluate_lazy(train_df.describe())
#test_df.describe()

,building_id,meter,timestamp,meter_reading,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
count,500000.000000,500000.000000,500000,5.000000e+05,500000.000000,500000.000000,201500.000000,87264.000000,479988.000000,225597.000000,479935.000000,395762.000000,426745.000000,457575.000000,479380.000000
mean,812.958450,0.657192,2016-01-05 12:18:30.218400,1.989620e+03,8.155352,107935.531148,1967.206655,4.179685,4.667253,1.979135,-0.761307,0.370139,1018.520594,173.341253,3.308224
min,0.000000,0.000000,2016-01-01 00:00:00,0.000000e+00,0.000000,283.000000,1900.000000,1.000000,-17.200000,0.000000,-22.800000,-1.000000,974.400000,0.000000,0.000000
25%,413.000000,0.000000,2016-01-03 06:00:00,1.210000e+01,3.000000,32256.000000,1951.000000,1.000000,-0.600000,0.000000,-6.700000,0.000000,1013.800000,70.000000,2.100000
50%,907.000000,0.000000,2016-01-05 12:00:00,6.190000e+01,9.000000,72709.000000,1968.000000,3.000000,5.600000,0.000000,-0.600000,0.000000,1018.900000,170.000000,3.100000
75%,1196.000000,1.000000,2016-01-07 19:00:00,2.157640e+02,13.000000,140092.000000,1991.000000,6.000000,10.000000,4.000000,5.600000,0.000000,1023.500000,290.000000,4.600000
max,1448.000000,3.000000,2016-01-10 01:00:00,8.243400e+06,15.000000,875000.000000,2017.000000,26.000000,28.300000,9.000000,21.100000,66.000000,1039.000000,360.000000,14.000000
std,426.375596,0.933593,NaN,9.207137e+04,5.101866,117335.376887,30.073146,4.007954,7.924662,2.855376,8.579566,3.124814,9.717889,122.448751,2.285083


In [15]:
########To check the Missing values count by columns wise#################
print(evaluate_lazy(train_df.isnull().sum()))
print(evaluate_lazy(test_df.isnull().sum()))


building_id                0
meter                      0
timestamp                  0
meter_reading              0
site_id                    0
primary_use                0
square_feet                0
year_built            298500
floor_count           412736
air_temperature        20012
cloud_coverage        274403
dew_temperature        20065
precip_depth_1_hr     104238
sea_level_pressure     73255
wind_direction         42425
wind_speed             20620
dtype: int64
row_id                     0
building_id                0
meter                      0
timestamp                  0
site_id                    0
primary_use                0
square_feet                0
year_built                 0
floor_count           500000
air_temperature            0
cloud_coverage        189755
dew_temperature            0
precip_depth_1_hr        774
sea_level_pressure     15093
wind_direction         15222
wind_speed                 0
dtype: int64


In [16]:
###########Missing value imputation for both train and test data sets manually ###############
train_df['floor_count'] = train_df['floor_count'].fillna(-999).astype(np.int16)
test_df['floor_count'] = test_df['floor_count'].fillna(-999).astype(np.int16)
train_df['year_built'] = train_df['year_built'].fillna(-999).astype(np.int16)
test_df['year_built'] = test_df['year_built'].fillna(-999).astype(np.int16)


In [17]:
train_df['air_temperature']=train_df['air_temperature'].fillna(-999).astype(np.int16)
train_df['cloud_coverage']=train_df['cloud_coverage'].fillna(-999).astype(np.int16)
train_df['dew_temperature']=train_df['dew_temperature'].fillna(-999).astype(np.int16)
train_df['precip_depth_1_hr']=train_df['precip_depth_1_hr'].fillna(-999).astype(np.int16)
train_df['sea_level_pressure']=train_df['sea_level_pressure'].fillna(-999).astype(np.int16)
train_df['wind_direction']=train_df['wind_direction'].fillna(-999).astype(np.int16)
train_df['wind_speed']=train_df['wind_speed'].fillna(-999).astype(np.int16)

test_df['air_temperature']=test_df['air_temperature'].fillna(-999).astype(np.int16)
test_df['cloud_coverage']=test_df['cloud_coverage'].fillna(-999).astype(np.int16)
test_df['dew_temperature']=test_df['dew_temperature'].fillna(-999).astype(np.int16)
test_df['precip_depth_1_hr']=test_df['precip_depth_1_hr'].fillna(-999).astype(np.int16)
test_df['sea_level_pressure']=test_df['sea_level_pressure'].fillna(-999).astype(np.int16)
test_df['wind_direction']=test_df['wind_direction'].fillna(-999).astype(np.int16)
test_df['wind_speed']=test_df['wind_speed'].fillna(-999).astype(np.int16)

In [18]:
print(evaluate_lazy(train_df.isnull().sum()))

building_id           0
meter                 0
timestamp             0
meter_reading         0
site_id               0
primary_use           0
square_feet           0
year_built            0
floor_count           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64
